In [1]:
import numpy as np
import matplotlib
from mido import Message, MetaMessage
import mido
import sys
import random
import os
import time
import math
pwd = "/home/zedroman/Documents/Sonia_Data/Songs/bethoven_sample/"
savepwd="/home/zedroman/Documents/Sonia_Data/Songs/_test/"
music = sorted([f for f in os.listdir(pwd) if f[len(f)-4:len(f)] == ".mid"])
sos = 128
message_type = {"note_on":1, "note_off":0}
type_message = {1:"note_on", 0:"note_off"}
tempo = 500000 * 5
hidden_size = 512
classes = 129
max_len = 100

velocity = 60
time_range = 16;
tempo = 5000000
pedal_max = 10

decoder_params = {velocity:velocity, time_range:time_range, tempo:tempo,pedal_max:pedal_max}

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device="cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def create_random(music):
    randoms = []
    for i in range(len(music)):
        r = []
        for j in range(3):
            r.append([random.random() * 10 for i in range(hidden_size)])
        randoms.append(r)
    return randoms
random_starts = create_random(music)

In [4]:

def midi_to_list(mid=None, pwd=None, tensor=True):
    if(mid==None):
          mid = mido.MidiFile(pwd)
    sample = []
    for i, track in enumerate(mid.tracks):
        for msg in track:
            if (msg.type == "note_on"):
                sample.append(msg.note)
    if (tensor):
        return torch.tensor(sample, device=device)
    else:
        return sample

def list_to_midi(sample):
    midi = mido.MidiFile(type=0)
    tr = mido.MidiTrack()
    midi.tracks.append(tr)
    tr.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    
    pedal = pedal_max
    last_notes = []
    
    for i in sample:
        if (pedal == 0):
            for j in range(len(last_notes) - 1):
                tr.append(Message("note_off", note=last_notes[j], velocity=velocity, time=0, channel=0))
            pedal = pedal_max
            last_notes = [last_notes[-1]]
        tr.append(Message("note_on", note=i, velocity=velocity, time=time_range, channel=0))
        pedal -= 1
        last_notes.append(i)
    for i in last_notes:
        tr.append(Message("note_off", note=i, velocity=velocity, time=0, channel=0))
    return midi

In [5]:
for file in (music):
    midi_file = mido.MidiFile(pwd + file)
    sample = midi_to_list(mid=midi_file, tensor=False)
#     print(sample)
    print(file)
#     for track in midi.tracks:
#         for msg in track:
#             print(msg)
    midi = list_to_midi(sample)
    midi.save("/home/zedroman/Documents/Sonia_Data/Songs/_test/test_to.mid")
    break

0.mid


In [6]:

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [7]:

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, other):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru1 = nn.GRU(classes, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)
        self.gru3 = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        self.velocity = other[velocity]
        self.time_range = other[time_range]
        self.tempo = other[tempo]
        self.pedal_max = other[pedal_max]

    def forward(self, input, hidden):
        output, hidden[0] = self.gru1(input, hidden[0])
        output = F.relu(output)
        output, hidden[1] = self.gru2(output, hidden[1])
        output = F.relu(output)
        output, hidden[2] = self.gru3(output, hidden[2])
        output = self.softmax(self.out(output[0]))
        
        return output, hidden

In [8]:

def one_hot(c):
    x = torch.zeros(classes, device=device, dtype=torch.float)
    x[c] = 1;
    return x.view(1,1,-1)

def train(target_tensor, decoder, decoder_optimizer, criterion, max_length=max_len, num = -1):
    decoder_optimizer.zero_grad()
    target_length = target_tensor.size(0)

    loss = 0
    
    decoder_input = one_hot(sos)
    decoder_hidden = [torch.tensor(random_starts[num][0], device=device, dtype=torch.float).view(1,1,-1),
                        torch.tensor(random_starts[num][1], device=device, dtype=torch.float).view(1,1,-1),
                        torch.tensor(random_starts[num][2], device=device, dtype=torch.float).view(1,1,-1)]

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di:di+1])
        decoder_input = one_hot(target_tensor[di])  # Teacher forcing

    loss.backward()

    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
def trainIters(decoder, n_iters,decoder_optimizer, print_every=len(music), epoch=0):
    start = time.time()
    print_loss_total = 0  # Reset every print_every
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = music[iter % (len(music))]
        midi_file = mido.MidiFile(pwd + training_pair)
        target_tensor = midi_to_list(mid=midi_file)
        loss = train(target_tensor,
                     decoder, decoder_optimizer, criterion, num = iter % (len(music)))
        print_loss_total += loss

        
        if iter % print_every == 0:
            last_loss = print_loss_total / print_every
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [11]:
# learning_rate = 0.001
# decoder = DecoderRNN(hidden_size, classes, decoder_params).to(device)
# decoder_optimizer = optim.Adam(decoder.parameters() , lr=learning_rate)

In [25]:
trainIters(decoder, 10000, decoder_optimizer)

0m 15s (- 238m 50s) (11 0%) 0.1224
0m 31s (- 237m 39s) (22 0%) 0.1074
0m 47s (- 236m 57s) (33 0%) 0.0955
1m 2s (- 236m 25s) (44 0%) 0.0857
1m 18s (- 236m 4s) (55 0%) 0.0780
1m 33s (- 235m 45s) (66 0%) 0.0715
1m 49s (- 235m 30s) (77 0%) 0.0672
2m 5s (- 235m 14s) (88 0%) 0.0647
2m 20s (- 234m 55s) (99 0%) 0.0630
2m 36s (- 234m 40s) (110 1%) 0.0608
2m 52s (- 234m 19s) (121 1%) 0.0569
3m 7s (- 234m 0s) (132 1%) 0.0517
3m 23s (- 233m 40s) (143 1%) 0.0488
3m 39s (- 233m 24s) (154 1%) 0.0446
3m 54s (- 233m 8s) (165 1%) 0.0430
4m 10s (- 232m 54s) (176 1%) 0.0429
4m 26s (- 232m 39s) (187 1%) 0.0405
4m 41s (- 232m 26s) (198 1%) 0.0384
4m 57s (- 232m 24s) (209 2%) 0.0373
5m 14s (- 232m 54s) (220 2%) 0.0335
5m 30s (- 232m 54s) (231 2%) 0.0308
5m 47s (- 233m 23s) (242 2%) 0.0293
6m 4s (- 233m 50s) (253 2%) 0.0276
6m 21s (- 234m 28s) (264 2%) 0.0260
6m 38s (- 234m 54s) (275 2%) 0.0256
6m 55s (- 235m 12s) (286 2%) 0.0248
7m 11s (- 235m 11s) (297 2%) 0.0240
7m 28s (- 235m 18s) (308 3%) 0.0235
7m 45s (

KeyboardInterrupt: 

In [27]:
def evaluate(decoder,num=-1, max_length=max_len):
    with torch.no_grad():

        decoder_input = one_hot(sos)
        decoder_hidden = []
        for j in range(3):
            decoder_hidden.append([random.random() * 10 for i in range(hidden_size)])
        decoder_hidden =   [torch.tensor(decoder_hidden[0], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[1], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[2], device=device).view(1,1,-1)]

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(3)
            topv = topv.max().item() - topv.cpu()[0].numpy()
            summ = topv.sum()
            chosen = np.random.choice(topi[0].cpu(), p= topv / summ)
            decoder_input = one_hot(chosen)
            decoded_words.append(chosen)

        return decoded_words
    
def generate(number, decoder):
    for i in range(number):
        new_music = evaluate(decoder)
        midi = list_to_midi(new_music)
        print(midi.length)
        midi.save(savepwd +"test" + str(i) + ".mid")
    return midi

last = generate(10, decoder)
for track in last.tracks:
    for msg in track:
        print(msg)

16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
<meta message set_tempo tempo=5000000 time=0>
note_on channel=0 note=64 velocity=60 time=16
note_on channel=0 note=60 velocity=60 time=16
note_on channel=0 note=79 velocity=60 time=16
note_on channel=0 note=58 velocity=60 time=16
note_on channel=0 note=63 velocity=60 time=16
note_on channel=0 note=79 velocity=60 time=16
note_on channel=0 note=46 velocity=60 time=16
note_on channel=0 note=77 velocity=60 time=16
note_on channel=0 note=84 velocity=60 time=16
note_on channel=0 note=84 velocity=60 time=16
note_off channel=0 note=64 velocity=60 time=0
note_off channel=0 note=60 velocity=60 time=0
note_off channel=0 note=79 velocity=60 time=0
note_off channel=0 note=58 velocity=60 time=0
note_off channel=0 note=63 velocity=60 time=0
note_off channel=0 note=79 velocity=60 time=0
note_off channel=0 note=46 v

In [28]:
torch.save({
            'model_state_dict': decoder.state_dict(),
            'optimizer_state_dict': decoder_optimizer.state_dict(),
            'random_starts' : random_starts
            }, '/home/zedroman/Documents/Sonia/models/bethoven')

In [20]:
decoder2 = DecoderRNN(hidden_size, classes, decoder_params).to(device)
decoder_optimizer2 = optim.Adam(decoder.parameters() , lr=learning_rate)
checkpoint = torch.load('/home/zedroman/Documents/Sonia/models/bethoven')
decoder2.load_state_dict(checkpoint['model_state_dict'])
random_starts2 = checkpoint['random_starts']
decoder_optimizer2.load_state_dict(checkpoint['optimizer_state_dict'])

In [21]:
print(random_starts2==random_starts)
print(decoder2==decoder)
print(decoder_optimizer2==decoder_optimizer)

True
False
False


In [24]:
generate(10, decoder2)

16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647
16.666666666666647


<midi file None type 0, 1 tracks, 201 messages>